In [153]:
from bs4 import BeautifulSoup
import urllib.request
import collections
import re
import json
import ast
import pandas as pd

In [147]:
def getSoup(url):
    #url="http://www.gnc.com/CytoSport-Monster-Milk/product.jsp?productId=99487386&cp=88200676.106012736"
    content = urllib.request.urlopen(url)
    soup = BeautifulSoup(content, "html.parser")
    return soup

In [148]:
def getServingPerContainer(soup):
    packageInfo = soup.find_all('td')
    servingSize=""
    servingPerContainer=""
    for product in packageInfo:
        #print(str(s.span)) 
        if str(product.span) == "<span>Serving Size</span>":
            #print(s.contents[1])
            servingSize = product.contents[1]
        if str(product.span) == "<span>Servings Per Container</span>":
            servingPerContainer = product.contents[1]
            servingPerContainer = servingPerContainer.replace(" ", "")
    return servingPerContainer

        

In [149]:
def removeUnit(value):
    #Removes the units (mg, g) and places it in the key value
    cleanUnit=""
    cleanValue=""
    cleanText = re.search("([0-9]+[\.]*[0-9]*)", value)
    if cleanText:
        cleanValue = cleanText.group(0)
    cleanUnitText = re.search("([0-9]+[\.]*[0-9]*) ([a-z]*)",value)
    if cleanUnitText:
        cleanUnit = cleanUnitText.group(2)
        cleanUnit = " " +cleanUnit
    return cleanValue, cleanUnit

In [150]:
def parseNutrition(soup):
    #
    key=""
    value=""
    #Parse out the GNC nutrition information
    nutrition = collections.OrderedDict()

    #Get Product Title
    productTitleScrape=soup.find_all("div",{"id":"product-title"})
    for title in productTitleScrape:
        temp = title.find('h2')
        if temp:
            nutrition['productTitle'] = temp.text

    #Get product Price
    productPriceScrape = soup.find_all("div",{"class":"promo goldCard member-price"})
    memberPrice = productPriceScrape[1].text
    p = re.search("\$([0-9]+[\.]*[0-9]*)", memberPrice)
    memberPrice = p.groups()[0]
    nutrition['memberPrice']= memberPrice
    
    #Get Servings Per Container 
    nutrition['NumBottles'] = getServingPerContainer(soup)

    #get Nutrition Information
    tr = soup.find_all("tr",{"class":"group-nutrition-elements "})
    for row in tr:
        cells = row.findAll("td")
        title = cells[0].string
        field = cells[1].string
        key = title + removeUnit(field)[1]
        nutrition[key] = removeUnit(value)[0]
        

    tr = soup.find_all("tr",{"class":"group-ingredients "})
    for row in tr:
        cells = row.findAll("td")
        title = cells[0].string
        value = cells[1].string
        value = removeUnit(value)[0]
        key = title + removeUnit(value)[1]
        nutrition[key] = removeUnit(value)[0]
    
    tr = soup.find_all("tr",{"class":"group-ingredients first"})
    for row in tr:
        cells = row.findAll("td")
        title = cells[0].string
        value = cells[1].string
        value = removeUnit(value)[0]
        key = title + removeUnit(value)[1]
        nutrition[key] = removeUnit(value)[0]

    tr = soup.find_all("tr",{"class":"group-nutrition-elements first"})
    for row in tr:
        cells = row.findAll("td")
        title = cells[0].string
        value = cells[1].string
        value = removeUnit(value)[0]
        key = title + removeUnit(value)[1]
        nutrition[key] = removeUnit(value)[0]


    #create df
    df2 = pd.DataFrame(nutrition, index=[0])

    return df2


In [151]:
def grabURLs(hyperlink):
    soup2 = getSoup(hyperlink)
    urlList = []
    for dt in soup2.find_all("dt"):
        for link in dt.find_all("a"):
            url = "http://www.gnc.com" + link.get('href')
            urlList.append(url)
    return urlList

        

In [154]:
#Get the Protein Drink Category
df=pd.DataFrame()
df2=pd.DataFrame()
hyperlink = "http://www.gnc.com/Food-Drink/Drinks/Protein-Drinks/family.jsp?categoryId=97952756"
urlList = grabURLs(hyperlink)
for url in urlList:
    soup = getSoup(url)
    df2 = parseNutrition(soup)
    df = df.append(df2, ignore_index=True)